In [ ]:
import os
import pathlib
from io import StringIO

import sqlalchemy as sa
from dotenv import dotenv_values, load_dotenv
from google.cloud import secretmanager
from google.cloud.sql.connector import Connector
from sqlalchemy.orm import DeclarativeBase, Session

In [ ]:
client = secretmanager.SecretManagerServiceClient()
project_id = "811358834395"
secret_id = "python-wifi-api"
version_id = "1"
resource_name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
response = client.access_secret_version(request={"name": resource_name})
payload = response.payload.data.decode("UTF-8")
print(payload)

In [ ]:
config = load_dotenv(stream=StringIO(payload), override=True)
print(config)
env = dotenv_values(stream=StringIO(payload))
for k, v in env.items():
    print(k, "=", v)

In [ ]:
db_instance = os.getenv("PSQL_INSTANCE")
db_username = os.getenv("PSQL_USERNAME")
db_password = os.getenv("PSQL_PASSWORD")
db_database = os.getenv("PSQL_DATABASE")

print(db_instance, db_username, db_password, db_database)

In [ ]:
connector = Connector()


def getConn(connector=connector):
    return connector.connect(
        db_instance,
        "pg8000",
        user=db_username,
        password=db_password,
        db=db_database,
        # ip_type = "private",
    )

In [ ]:
engine = sa.create_engine(
    "postgresql+pg8000://",
    creator=getConn,
)

select_query = """
    SELECT DISTINCT ON (hospital_id) path, hospital_id, id
FROM fp_models 
WHERE is_active = true
    """

with engine.connect() as db_conn:
    # 1. Find all records with the same hospital_id and is_active = true

    active_rows = db_conn.execute(sa.text(select_query)).fetchall()

print(active_rows)

In [ ]:
class Base(DeclarativeBase):
    pass

class FpModel(Base):
    __tablename__ = "fp_models"
    id = sa.Column(sa.Integer, primary_key=True)
    name = sa.Column(sa.String)
    path = sa.Column(sa.String)

In [ ]:
session = Session(engine)

In [ ]:
stmt = sa.select(FpModel)
for fp in session.scalars(stmt):
    print(fp.id, fp.name, fp.path)